In [ ]:
# default_exp models.siren

# SiReN
> Sign-Aware Recommendation Systems with Graph Neural Networks (SiReN).

SiRen first constructs a signed bipartite graph $\mathcal{E}^s = (u,v,w^s_{uv})|w^s_{uv} = w_{uv} − w_o,(u,v,w_{uv}) ∈ \mathcal{E}$. Then it split this into 2 graphs. $\mathcal{E}^p = (u,v,w^s_{uv})|w^s_{uv} > 0,\  (u,v,w^s_{uv}) ∈ \mathcal{E}^s$, and $\mathcal{E}^n = (u,v,w^s_{uv})|w^s_{uv} < 0,\ (u,v,w^s_{uv}) ∈ \mathcal{E}^s$. The purpose of this graph partitioning is to make the graphs $G^p$ and $G^n$, respectively, assortative and disassortative so that each partitioned graph is used as input to the most appropriate learning model.

For assortative relation learning, we use GNN network and for learning the disassortative relations, MLP Network is a better candidate. The resultant embeddings $Z^p$ and $Z^n$ are then reweighted using attention mechanism. Finally, the whole network parameters is optimized using a sign-aware BPR loss function.

Note: For the graph with negative edges, we adopt a multi-layer perceptron (MLP) due to the fact that negative edges can weaken the homophily and thus message passing to such dissimilar nodes would not be feasible.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import torch
from torch import nn
import torch.nn.functional as F
from torch_geometric.data import Data

from recohut.models.layers.message_passing import LightGConv, LRGCCF

In [ ]:
#export
class SiReN(nn.Module):
    def __init__(self,
                 train,
                 num_u,
                 num_v,
                 offset,
                 num_layers = 2,
                 MLP_layers = 2,
                 dim = 64,
                 reg = 1e-4,
                 device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
                 graph_enc = 'lightgcn',
                 user_col = 'userId',
                 item_col = 'itemId',
                 rating_col = 'rating'):
        super(SiReN,self).__init__()
        self.M = num_u; self.N = num_v;
        self.num_layers = num_layers
        self.MLP_layers = MLP_layers
        self.device = device
        self.reg = reg
        self.embed_dim = dim
        edge_user = torch.tensor(train[train[rating_col]>offset][user_col].values-1)
        edge_item = torch.tensor(train[train[rating_col]>offset][item_col].values-1)+self.M
        edge_ = torch.stack((torch.cat((edge_user,edge_item),0),torch.cat((edge_item,edge_user),0)),0)
        self.data_p=Data(edge_index=edge_)
        # For the graph with positive edges
        self.E = nn.Parameter(torch.empty(self.M + self.N, dim))
        nn.init.xavier_normal_(self.E.data)
        self.convs = nn.ModuleList()
        self.mlps = nn.ModuleList()
        for _ in range(num_layers):
            if graph_enc == 'lightgcn':
                self.convs.append(LightGConv()) 
            else:
                self.convs.append(LRGCCF(dim,dim))
        # For the graph with negative edges
        self.E2 = nn.Parameter(torch.empty(self.M + self.N, dim))
        nn.init.xavier_normal_(self.E2.data)
        for _ in range(MLP_layers):
            self.mlps.append(nn.Linear(dim,dim,bias=True))
            nn.init.xavier_normal_(self.mlps[-1].weight.data)
        # Attntion model
        self.attn = nn.Linear(dim,dim,bias=True)
        self.q = nn.Linear(dim,1,bias=False)
        self.attn_softmax = nn.Softmax(dim=1)
        
    def aggregate(self):
        # Generate embeddings z_p
        B=[]; B.append(self.E)
        x = self.convs[0](self.E,self.data_p.edge_index)
        B.append(x)
        for i in range(1,self.num_layers):
            x = self.convs[i](x,self.data_p.edge_index)
            B.append(x)
        z_p = sum(B)/len(B) 
        # Generate embeddings z_n
        C = []; C.append(self.E2)
        x = F.dropout(F.relu(self.mlps[0](self.E2)),p=0.5,training=self.training)
        for i in range(1,self.MLP_layers):
            x = self.mlps[i](x);
            x = F.relu(x)
            x = F.dropout(x,p=0.5,training=self.training)
            C.append(x)
        z_n = C[-1]
        # Attntion for final embeddings Z
        w_p = self.q(F.dropout(torch.tanh((self.attn(z_p))),p=0.5,training=self.training))
        w_n = self.q(F.dropout(torch.tanh((self.attn(z_n))),p=0.5,training=self.training))
        alpha_ = self.attn_softmax(torch.cat([w_p,w_n],dim=1))
        Z = alpha_[:,0].view(len(z_p),1) * z_p + alpha_[:,1].view(len(z_p),1) * z_n
        return Z
    
    def forward(self,u,v,w,n,device):
        emb = self.aggregate()
        u_ = emb[u].to(device);
        v_ = emb[v].to(device);
        n_ = emb[n].to(device);
        w_ = w.to(device)
        positivebatch = torch.mul(u_ , v_ ); 
        negativebatch = torch.mul(u_.view(len(u_),1,self.embed_dim),n_)  
        sBPR_loss =  F.logsigmoid((torch.sign(w_).view(len(u_),1) * (positivebatch.sum(dim=1).view(len(u_),1))) - negativebatch.sum(dim=2)).sum(dim=1)
        reg_loss = u_.norm(dim=1).pow(2).sum() + v_.norm(dim=1).pow(2).sum() + n_.norm(dim=2).pow(2).sum();
        return -torch.sum(sBPR_loss) + self.reg * reg_loss

In [ ]:
import pandas as pd

train = pd.DataFrame(
    {'userId':[1,1,2,2,3,4,5],
     'itemId':[1,2,1,3,2,4,5],
     'rating':[4,5,2,5,3,2,4]}
)

train

,userId,itemId,rating
0,1,1,4
1,1,2,5
2,2,1,2
3,2,3,5
4,3,2,3
5,4,4,2
6,5,5,4


In [ ]:
model = SiReN(train,
                num_u = 5,
                num_v = 5,
                offset = 3.5,
                num_layers = 1,
                MLP_layers = 1,
                dim = 2,
                reg = 1e-4,
                device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
                graph_enc = 'lightgcn',
                user_col = 'userId',
                item_col = 'itemId',
                rating_col = 'rating')

In [ ]:
torch.random.manual_seed(0)
model.aggregate()

tensor([[ 0.3599, -0.6117],
        [ 0.0643,  0.1232],
        [-0.0828,  0.1913],
        [-0.3257, -0.1678],
        [ 0.2391, -0.0063],
        [-0.0721, -0.3664],
        [-0.0213, -0.1475],
        [ 0.1260, -0.0493],
        [-0.0176, -0.1874],
        [ 0.0924,  0.2289]], grad_fn=<AddBackward0>)

In [ ]:
#hide
!pip install -q watermark
%reload_ext watermark
%watermark -a "Sparsh A." -m -iv -u -t -d -p torch_geometric

Author: Sparsh A.

Last updated: 2021-12-20 05:06:04

torch_geometric: 2.0.2

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.4.104+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

pandas : 1.1.5
torch  : 1.10.0+cu111
IPython: 5.5.0

